In [1]:
%pylab inline
%matplotlib inline

import os
import time
import collections
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")
sys.path.append("/Users/Bya/git/predictEPL/WebScrapping/")

import paths
import espn_urls
import my_plot
import scrap_espn_gamecast
import emolex
import useful_methods
import tokenizer

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

dfGameInfos.head()

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time
0,1,Tottenham,Sat. 8 Aug.,United,0,1,0,1,11:45
1,1,Watford,Sat. 8 Aug.,Everton,2,2,1,0,14:00
2,1,Sunderland,Sat. 8 Aug.,Leicester,2,4,0,3,14:00
3,1,Crystal,Sat. 8 Aug.,Norwich,3,1,1,0,14:00
4,1,Villa,Sat. 8 Aug.,Bournemouth,1,0,0,0,14:00


In [6]:
# HT: Equal games
# dfGameInfosEqual = dfGameInfos[(dfGameInfos['score_ht_home'] == dfGameInfos['score_ht_away']) &
#                                (dfGameInfos['GW'] >= 4)].copy()

weeks = list(dfGameInfos['GW'])
home_teams = list(dfGameInfos['home_team'])
away_teams = list(dfGameInfos['away_team'])
score_ht_homes = list(dfGameInfos['score_ht_home'])
score_ht_aways = list(dfGameInfos['score_ht_away'])
score_ft_homes = list(dfGameInfos['score_ft_home'])
score_ft_aways = list(dfGameInfos['score_ft_away'])

In [3]:
# Read Emotion-Lexicon-Soccer as Dictionary
dic_emolex_soccer, dic_emolex_stemmed_soccer = emolex.EmolexSoccerDic()

All Words: 14163


In [4]:
def EmolexPN(df, start=1, end=63):
    dfEmolex = df.copy()
    
    # set interval
    ith_minute_start = start
    ith_minute_end = end

    # convert to integer
    dfEmolex['ith_minute'] = [int(number) for number in dfEmolex['ith_minute']]
    dfEmolex['positive'] = [int(number) for number in dfEmolex['positive']]
    dfEmolex['negative'] = [int(number) for number in dfEmolex['negative']]

    # Minute Interval's DF
    dfEmolex = dfEmolex[
        (dfEmolex['ith_minute'] >= ith_minute_start) &
        (dfEmolex['ith_minute'] <= ith_minute_end)
    ].copy()

    # Counting pos, neg
    positive = sum(list(dfEmolex['positive']))
    negative = sum(list(dfEmolex['negative']))
    
    # Return Positve Percentage
    return (positive / (positive + negative))

In [8]:
res_ht_rt = []

start_time = time.time()

for index in range(len(weeks)):
    try:
        # Read Single Game as DF
        filename = home_teams[index] + '_vs_' + away_teams[index] + '.csv'
        os.chdir(paths.READ_PATH_EXTRACTED_CSV + 'GW' + str(weeks[index]) + '/SingleGames/')
        df = useful_methods.csv_dic_df(filename)

        # Filter DF: remove tweets of stream, bots ...
        dfFilter = useful_methods.FilterDF(df)

        # Count Emolex categories on Tweets by minutes
        dfFilterEmolexSoccerHome = emolex.CreateEmolexDF(dfFilter[dfFilter['side'] == 'home'], dic_emolex_soccer, dic_emolex_stemmed_soccer)
        dfFilterEmolexSoccerAway = emolex.CreateEmolexDF(dfFilter[dfFilter['side'] == 'away'], dic_emolex_soccer, dic_emolex_stemmed_soccer)

        # Calculate PN %
        emolex_pn_home = EmolexPN(dfFilterEmolexSoccerHome)
        emolex_pn_away = EmolexPN(dfFilterEmolexSoccerAway)

        # Append Resulst
        res_ht_rt.append([index, emolex_pn_home, emolex_pn_away])
    except FileNotFoundError:
        continue

print("[Done]: %.2f" % (time.time() - start_time))

[Done]: 1719.13


In [9]:
ok = 0
ng = 0
eq = 0

for res in res_ht_rt:
    index, pn_home, pn_away = res
    
    if (pn_home > pn_away) and (score_ft_homes[index] > score_ft_aways[index]):
        ok += 1
        print("\n------------------------------")
        print("[GAME]: GW%s %s vs %s" %(weeks[index], home_teams[index], away_teams[index]))
        print("[SCORE]: HT(%s-%s) => FT(%s-%s)" % (score_ht_homes[index], score_ht_aways[index],
                                          score_ft_homes[index], score_ft_aways[index]))
        print("\t[Emolex PN Score]: %s vs %s" % (pn_home, pn_away))

    elif (pn_home < pn_away) and (score_ft_homes[index] < score_ft_aways[index]):
        ok += 1
        print("\n------------------------------")
        print("[GAME]: GW%s %s vs %s" %(weeks[index], home_teams[index], away_teams[index]))
        print("[SCORE]: HT(%s-%s) => FT(%s-%s)" % (score_ht_homes[index], score_ht_aways[index],
                                          score_ft_homes[index], score_ft_aways[index]))
        print("[Emolex PN Score]: %s vs %s" % (pn_home, pn_away))
    
    elif score_ft_homes[index] == score_ft_aways[index]:
        eq += 1
    else:
        ng += 1

print("\n\n\nOK: %s, NG: %s, EQ: %s" % (ok, ng, eq))


------------------------------
[GAME]: GW4 Newcastle vs Arsenal
[SCORE]: HT(0-0) => FT(0-1)
[Emolex PN Score]: 0.36715527078 vs 0.468262348938

------------------------------
[GAME]: GW4 Stoke vs WestBromwich
[SCORE]: HT(0-1) => FT(0-1)
[Emolex PN Score]: 0.138181818182 vs 0.482456140351

------------------------------
[GAME]: GW4 Liverpool vs WestHam
[SCORE]: HT(0-2) => FT(0-3)
[Emolex PN Score]: 0.543828441546 vs 0.834787667248

------------------------------
[GAME]: GW4 Chelsea vs Crystal
[SCORE]: HT(0-0) => FT(1-2)
[Emolex PN Score]: 0.600338218715 vs 0.828409090909

------------------------------
[GAME]: GW4 City vs Watford
[SCORE]: HT(0-0) => FT(2-0)
	[Emolex PN Score]: 0.760869565217 vs 0.674074074074

------------------------------
[GAME]: GW4 Southampton vs Norwich
[SCORE]: HT(1-0) => FT(3-0)
	[Emolex PN Score]: 0.593553459119 vs 0.484293193717

------------------------------
[GAME]: GW4 Swansea vs United
[SCORE]: HT(0-0) => FT(2-1)
	[Emolex PN Score]: 0.815523657629 vs 0.644

In [10]:
# Save Resulst as CSV
dfResult = dfGameInfos.copy()

def PnScore(index, side):
    for res in res_ht_rt:
        if index == res[0]:
            return res[side]
    return -1

dfResult['pn_away'] = [PnScore(index, 2) for index in range(len(weeks))]
dfResult['pn_home'] = [PnScore(index, 1) for index in range(len(weeks))]
    

# Save as CSV
useful_methods.DFtoCSV(dfResult, paths.READ_PATH_GAME_INFO, 'Predict-Emolex-63min', index=False)
print("[Saved in]: %s" % (paths.READ_PATH_GAME_INFO + 'Predict-Emolex-63min.csv'))

[Saved in]: /Users/Bya/Dropbox/Research/datas/EPL/Predict-Emolex-63min.csv
